In [15]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import random


In [2]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [5]:

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content


In [16]:
def get_genre(animal):
    print(f"Tool function is called for {animal}")
    genres = ["tribal with martial arts background", "symmetric with dark and evil look", "as a boxer", "as a jiu jitsu practitioner", "in terms of david vs goliath  setting"]
    selected_genre = random.choice(genres)
    return selected_genre


In [8]:
# There's a particular dictionary structure that's required to describe our function:

genre_function = {
    "name": "get_genre",
    "description": "Get the specific idea of the tattoo depending on the animal. Call this whenever you need to create a tattoo of an animal, for example when a customer asks 'I would like to create a tattoo of a honey-badger'",
    "parameters": {
        "type": "object",
        "properties": {
            "animal": {
                "type": "string",
                "description": "The animal for the tattoo",
            },
        },
        "required": ["animal"],
        "additionalProperties": False
    }
}

In [10]:


tools = [{"type": "function", "function": genre_function}]

In [30]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    print(f"history = {history}")
    print(f"Message = {message}")
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, animal = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    print(messages)

    
    return response.choices[0].message.content

In [43]:


def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    animal = arguments.get('animal')
    genre = get_genre(animal)
    response = {
        "role": "tool",
        "content": json.dumps({"animal": animal,"genre": genre}),
        "tool_call_id": tool_call.id
    }
    return response, animal,genre

In [44]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [45]:
def artist(animal, genre):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a tattoo of a {animal}, in a style representing {genre} in a cool way",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [46]:
!ffmpeg -version
!ffprobe -version
!ffplay -version

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
built with Apple clang version 16.0.0 (clang-1600.0.26.4)
configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

In [38]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",   
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [47]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, animal, genre = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(animal,genre)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    talker(reply)
    
    return history, image

In [48]:


with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


Input #0, wav, from '/var/folders/71/dgh1mf3j6c1366wy9sdlj24r0000gn/T/tmpc_1are1j.wav':
  Duration: 00:00:02.04, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.90 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 


Tool function is called for badger


Input #0, wav, from '/var/folders/71/dgh1mf3j6c1366wy9sdlj24r0000gn/T/tmp4afvwm_u.wav':
  Duration: 00:00:05.09, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   5.01 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 